<a href="https://colab.research.google.com/github/denniesbor/spwio/blob/from_colab/ghosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import tqdm
import pickle
import pprint
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Set the data and working folders
working_dir = "/content/drive/MyDrive/spwio"
data_path = os.path.join(working_dir, "data")

In [6]:
# Read the US industry gross output and technical coefficients

technical_coefficients = pd.read_csv(os.path.join(data_path, "direct_requirements.csv")) # total requirements table

# 2022 US gross output by industires, in million dollars
gross_output_2022 = pd.read_csv(os.path.join(data_path, "use_table.csv"))
gross_output_2022.set_index("Name", inplace=True)

# Total by industry
industry_output = gross_output_2022.loc["Total industry output (basic prices)"].to_numpy().T.astype(int) /1000 # y output

print(industry_output)

[ 644.755  830.937  665.432 2131.939 7162.401 2481.375 2404.055 1771.549
 2406.922 8268.954 5338.993 3380.27  1832.25   871.147 4936.725]


## Ghosh Model

Ghosh model estimates how a change in industries input affects the output of the forward linked industrial sectors.
Ghosh model is given by:

$x' = v'G$ where G is

$G = (I-B)^{-1}$

B is derived from technical coefficients and industries gross output by

$B = \hat{X}^{-1} A \hat{X}$

The hat symbol matrix X consist of industral outputs as diagonal elements.

In [7]:
# Computing A matrix
# Import total requirements matrix
leontief_df = pd.read_csv(os.path.join(data_path, "total_requirements.csv"))

# Identity matrix
I = np.eye(leontief_df.shape[1])
A = I - np.linalg.inv(leontief_df.to_numpy())

In [10]:
# Computing B matrix
gross_diag = np.diag(industry_output) # Make the industrial output as the diagonal in a n by n matrix where n is number of industries/commodities
gross_diag_inv = np.linalg.inv(gross_diag) # Inverse of the diagonal matrix of outputs
B = np.dot(np.dot(gross_diag_inv, A), gross_diag) # Get the B coefficient matrix

# Allocation coefficients
print(B[1])


[0.00260726 0.10376166 0.04155259 0.02739609 0.69657082 0.0016575
 0.00106581 0.00267864 0.00546284 0.0013009  0.00686356 0.00603553
 0.00478607 0.00235111 0.04659834]


In [11]:
# Ghosh model
ghosh_model = np.linalg.inv(I - B)

## Ghosh Model

We shall leverage the Ghosh Model to estimate changes in gross output due to changes in value added.
The GDP calculations are based on the final demand and VA, ignoring double counting in intra and inter-industry gross output. Proceeding in this section, we shall apply relative values to US GDP to estimate, for example, how much losses of the gross production are incurred for a change in value added.

In [15]:
# Load the US GDP by 15 NAICs sectors
us_gdp_df = pd.read_csv(os.path.join(data_path, "gdp_sector_us.csv"))
us_gdp_df

,Industry,gdp
0,"Agriculture, forestry, fishing, and hunting",179.3
1,Mining,301.3
2,Utilities,336.3
3,Construction,850.9
4,Manufacturing,2312.9
5,Wholesale trade,1118.8
6,Retail trade,1318.5
7,Transportation and warehousing,736.1
8,Information,1625.4
9,"Finance, insurance, real estate, rental, and l...",4557.7


In [24]:
# Convert GDP vals as numpy array
va = us_gdp_df.gdp.to_numpy()
va_transposed = np.transpose(va)

# Sum the US GDP (in billion dollars)
us_gdp_df.gdp.sum()

22591.4

In [38]:
# Apply a 5% impact on GDP
va_transposed_5_perc = 0.05 * va_transposed

# Change in Gross output is v' @ Ghosh
x_delta_5_perc =  va_transposed_5_perc @ ghosh_model

print(f"Change in gross output /n {x_delta_5_perc}")

# For a 5% on the US value added, the resulting change in gross output is
("Total change in output {}" .format(np.sum(x_delta_5_perc)))

Change in gross output /n [ 23.98662499  32.60243351  27.94407776  86.96478485 315.40736544
 106.94300203 110.7915299   75.16337799 132.8289166  361.13996549
 259.64858211 157.59143901  82.05547435  34.94001963 220.00278601]


'Total change in output 2028.0103796489088'

A 5% impact on GDP would amount to a loss of 2 T in US gross output.

**In testing the model, a 100% impact would lead to 100% loss in gross output. Therefore, mat multiplying the total US gdp by the ghosh model should be equal to the total gross output**

In [30]:
# A 100% impact/ total economic shutdown

x_delta_100_perc =  va_transposed @ ghosh_model

print(f"Change in gross output, 100% impact: \n {x_delta_100_perc}")
print("")
print(f"2022 US Gross output by sectors: \n {industry_output}")

Change in gross output, 100% impact: 
 [ 479.73249975  652.04867017  558.88155513 1739.29569699 6308.14730874
 2138.86004057 2215.83059794 1503.26755982 2656.57833206 7222.79930973
 5192.97164221 3151.82878012 1641.10948691  698.80039255 4400.05572028]

2022 US Gross output: 
 [ 644.755  830.937  665.432 2131.939 7162.401 2481.375 2404.055 1771.549
 2406.922 8268.954 5338.993 3380.27  1832.25   871.147 4936.725]


The slight discrepancy in values is because the GDP figures used in the calculation are chained to 2017 values figures, whereas the gross output are nominal values

In [34]:
np.sum(x_delta_100_perc) # Change in gross output

48672.249111573794

In [32]:
np.sum(industry_output)

45127.70399999999

## Estimating indirect downstream impacts (Backward Linkages) due to change in value added